In [2]:
from datasets import load_dataset

d:\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load full dataset of 1.9 million examples
full_dataset = load_dataset("json", data_files='./data/arxiv-metadata-oai-snapshot.json')

Using custom data configuration default-750e5b2bf677a1ac


100%|██████████| 1/1 [00:00<00:00, 45.38it/s]


Dataset json downloaded and prepared to C:\Users\Ashwin Prakash\.cache\huggingface\datasets\json\default-750e5b2bf677a1ac\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


In [4]:
# Filter examples containing papers from thes cs.AI category
dataset = full_dataset.filter(lambda example: 'cs.AI' in example['categories'])

Parameter 'function'=<function <lambda> at 0x000001524134EE58> of the transform datasets.arrow_dataset.Dataset.filter couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 2135/2135 [01:54<00:00, 18.64ba/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'],
        num_rows: 52192
    })
})

In [6]:
# Remove unwanted columns and keep only abstract and title
cols_to_remove = list(dataset['train'].features.keys())
cols_to_remove.remove('abstract')
cols_to_remove.remove('title')

dataset = dataset.remove_columns(cols_to_remove)
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract'],
        num_rows: 52192
    })
})

In [7]:
# Split the dataset
dataset = dataset['train'].train_test_split(test_size=0.1)

test_val = dataset['test'].train_test_split(test_size=0.5)
dataset['val'] = test_val['train']
dataset['test'] = test_val['test']

dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract'],
        num_rows: 46972
    })
    test: Dataset({
        features: ['title', 'abstract'],
        num_rows: 2610
    })
    val: Dataset({
        features: ['title', 'abstract'],
        num_rows: 2610
    })
})

In [8]:
# Save the processed dataset to disk
dataset.save_to_disk('arxiv_AI_dataset')